# Sklearn—GridSearch 调参函数

GridSearch是Sklearn里的一个调参函数。本文是对此函数的详细解释。

## 参数搜索

参数并非从estimators中直接学到的，可以通过设置一个参数搜索空间来找到最佳的cross-validation score。

通常示例包括的参数有：SVM分类器的中C、kernel和gamma，Lasso中的alpha等。

当构建一个estimator时，提供的参数可以以这种方式进行优化。更特别的是，可以使用如下的方法来给给定estimator的所有参数来找到对应的参数名和当前值：

  estimator.get_params()
这些参数称被提到：“超参数（hyperparameters）”，尤其在Bayesian learning中，它们与机器学习过程中的参数优化是有区别的。

一个这样的参数search包含：

    一个estimator(regressor/classifier)
    一个参数空间
    一个用于searching/sampling候选参数的方法
    一个cross-validation的scheme
    一个score function

这样的模型允许你指定有效的搜索参数策略，如下。在sklearn中，有两种通用方法进行sampling搜索候选参数：

    GridSearch: 暴力搜索所有参数组合

    RandomizedSearchCV: 在指定参数空间内抽样一部分候选参数


## GRIDSEARCHCV

grid search提供了GridSearchCV，相应的参数空间param_grid设置如下：

      param_grid = [
        {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
        {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
       ]

上例指定了两个要搜索的参数空间：一个是线性kernel，其中C值为[1,10,100,1000]；另一个则使用RBF kernel，对应的C值为[1,10,100,1000]，对应的gamma值为 [0.001, 0.0001].

GridSearchCV实例实现了通用的estimator API: 当在数据集的所有可能参数组合上进行”fitting”时，所有参数组都会被评测，并保留最优的参数组合。

[sklearn SVM详解](https://www.cnblogs.com/crawer-1/p/8870700.html)

## 随机参数优化

使用GridSearch进行参数搜索是目前最广泛使用的参数优化方法，还有另一些方法存在。RandomizedSearchCV实现了在参数上的随机搜索，每个设置都会以可能的参数值分布进行抽样。对比穷举法，它具有两个优势：

    1.budget的选择与参数个数和可能的值独立

    2.增加参数不会影响性能，不会降低效果

参数设定部分和GridSearchCV类似，使用一个字典表来进行参数抽样。另外，计算开销（computation budget）, 抽取的样本数，抽样迭代次数，可以由n_iter来指定。对于每个参数，都可以指定在可能值上的分布，或者是一个离散值列表（它可以被均匀采样）。

例如：

      [{‘C’: scipy.stats.expon(scale=100), 
      ‘gamma’: scipy.stats.expon(scale=.1), 
      ‘kernel’: [‘rbf’], 
      ‘class_weight’:[‘auto’, None]}]
      
这个例子使用scipy.stats模块，该模块包含了许多分布方法可以用来进行抽样，包括：指数分布（expon），gamma分布(gamma)，均匀分布（uniform），或randint分布。通常每个函数都可以提供一个rvs（随机变量抽样）方法进行抽样。

注意：

    scipy.stats的分布不允许以随机方式指定。作为替代，我们可以使用一个全局的numpy 随机态，它可以通过np.random.seed或np.random.set_state来设定。

    对于连续的参数，比如上面的C，指定一个连续的分布十分重要，它可以完全利用随机化（randomization）。这种情况下，增加n_iter将产生一个更好的搜索。


## 指定一个目标METRIC

缺省的，参数搜索会使用estimator的缺省score函数来评估参数设置。

其中，分类使用sklearn.metrics.accuracy_score，

回归使用sklearn.metrics.r2_score。

对于其它应用，可能需要使用一个可合适的scoring函数（例如：对于unbalanced分类问题，accuracy的score是不合适的）。

可选择的scoring函数可以通过GridSearchCV/RandomizedSearchCV以及其它CV工具类的scoring参数来设置。详见。

## 参数使用

    sklearn.model_selection.GridSearchCV(
        estimator, param_grid, scoring=None,  
        fit_params=None, n_jobs=1, iid=True, 
        refit=True, cv=None, verbose=0,
        pre_dispatch=‘2*n_jobs’, 
        error_score=’raise’, return_train_score=’warn’)

- estimator：所使用的分类器，比如：estimator=RandomForestClassifier(min_samples_split=100, min_samples_leaf=20, max_depth=8, max_features='sqrt', random_state=10)，并且传入除需要确定最佳的参数之外的其他参数。每个分类器都需要一个scoring参数或者score方法。

- param_grid：值为字典或列表，即需要最优化的参数的取值，param_grid =param_test1，param_test1 = {'n_estimators':range(10,71,10)}

- scoring：准确评价标准，默认为None（使用estimator的误差估计函数），这时需要使用score函数；或者如scoring='roc_auc'，根据所选模型不同，评价准则不同。

- cv：交叉验证参数，默认为None

- refit：默认为True，程序将会以交叉验证训练集得到的最佳参数，重新对所有可用的训练集与测试集进行，作为最终用于性能评估的最佳模型参数。即在搜索参数结束后，用最佳参数结果再次fit一遍全部数据集。

- iid:默认True,为True时，默认为各个样本fold概率分布一致，误差估计为所有样本之和，而非各个fold的平均。

- verbose：日志冗长度，int：冗长度，0：不输出训练过程，1：偶尔输出，>1：对每个子模型都输出。

- n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值。

- pre_dispatch：指定总共分发的并行任务数。当n_jobs大于1时，数据将在每个运行点进行复制，这可能导致OOM，而设置pre_dispatch参数，则可以预先划分总共的job数量，使数据最多被复制pre_dispatch次，进行预测的常用方法和属性

属性方法：

- `grid.fit(train_x, train_y)`：运行网格搜索

- `grid_scores_`：给出不同参数情况下的评价结果

- `best_params_`：描述了已取得最佳结果的参数的组合

- `best_score_`：成员提供优化过程期间观察到的最好的评分

```py
from sklearn.grid_search import GridSearchCV

随机森林的参数

tree_param_grid={'min_sample_split':list((3,6,9)),'n_estimators':list((10,50,100))
                 
grid=GridSearchCV(RandomForestRegressor(),param_grid=tree_param_grid,cv=5)
                 
grid.fit(x,y)
                 
print(grid.grid_scores_,grid.best_params_,grid.best_score_)
```

# 使用案例

## 案例1 - KNN-GridSearchCV

In [1]:
from sklearn.datasets import load_iris  # 自带的样本数据集
from sklearn.neighbors import KNeighborsClassifier  # 要估计的是knn里面的参数，包括k的取值和样本权重分布方式
import matplotlib.pyplot as plt  # 可视化绘图
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV  # 网格搜索和随机搜索

In [2]:
iris = load_iris()

X = iris.data  # 150个样本，4个属性
y = iris.target # 150个类标号

k_range = range(1, 31)  # 优化参数k的取值范围
weight_options = ['uniform', 'distance']  # 代估参数权重的取值范围。uniform为统一取权值，distance表示距离倒数取权值
# 下面是构建parameter grid，其结构是key为参数名称，value是待搜索的数值列表的一个字典结构
param_grid = {'n_neighbors':k_range, 'weights':weight_options}  # 定义优化参数字典，字典中的key值必须是分类算法的函数的参数名
print(param_grid)

{'n_neighbors': range(1, 31), 'weights': ['uniform', 'distance']}


In [3]:
# 定义分类算法。n_neighbors和weights的参数名称和param_grid字典中的key名对应
knn = KNeighborsClassifier()  

# ================================网格搜索=======================================
# 这里GridSearchCV的参数形式和cross_val_score的形式差不多，
# 其中param_grid是parameter grid所对应的参数
# GridSearchCV中的n_jobs设置为-1时，可以实现并行计算（如果你的电脑支持的情况下）
grid = GridSearchCV(estimator=knn, param_grid=param_grid)
grid.fit(X, y)
print('网格搜索-度量记录：',grid.cv_results_)  # 包含每次训练的相关信息

D:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


网格搜索-度量记录： {'mean_fit_time': array([0.00099635, 0.00066479, 0.00033323, 0.00099834, 0.00066725,
       0.00033259, 0.00066495, 0.00033228, 0.0006725 , 0.00033243,
       0.        , 0.00066519, 0.        , 0.        , 0.00066471,
       0.00066519, 0.00066487, 0.00066511, 0.00066447, 0.00066471,
       0.00033267, 0.00066511, 0.00033259, 0.00033387, 0.00031749,
       0.00033236, 0.00066535, 0.00066551, 0.00066503, 0.00066471,
       0.00033251, 0.        , 0.0006663 , 0.00033243, 0.00033243,
       0.00033251, 0.00033236, 0.00033259, 0.        , 0.        ,
       0.00066495, 0.00066479, 0.        , 0.00066487, 0.00033236,
       0.00033236, 0.00033299, 0.00033275, 0.00066527, 0.00132926,
       0.00033236, 0.        , 0.00033267, 0.00066527, 0.00033243,
       0.00033236, 0.00066479, 0.0003322 , 0.00033267, 0.00066447]), 'std_fit_time': array([8.48537942e-07, 4.70078344e-04, 4.71257962e-04, 1.32507737e-06,
       4.71841164e-04, 4.70358829e-04, 4.70190252e-04, 4.69909263e-04,
       

D:\Program Files (x86)\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [6]:
print('网格搜索-最佳度量值:',grid.best_score_)  # 获取最佳度量值
print('网格搜索-最佳参数：',grid.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('网格搜索-最佳模型：',grid.best_estimator_)  # 获取最佳度量时的分类器模型

网格搜索-最佳度量值: 0.9866666666666667
网格搜索-最佳参数： {'n_neighbors': 5, 'weights': 'uniform'}
网格搜索-最佳模型： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')


In [7]:
# 使用获取的最佳参数生成模型，预测数据
knn = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'], weights=grid.best_params_['weights'])  # 取出最佳参数进行建模
knn.fit(X, y)  # 训练模型
print(knn.predict([[3, 5, 4, 2]]))  # 预测新对象

[1]


## 案例2 - KNN- RandomizedSearchCV

In [10]:
# ==================================随机搜索===========================================
grid = RandomizedSearchCV(knn, param_grid, cv=10, scoring='accuracy', random_state=5)
grid.fit(X, y)
print('随机搜索-度量记录：',grid.cv_results_)  # 包含每次训练的相关信息


随机搜索-度量记录： {'mean_fit_time': array([0.00039868, 0.0005949 , 0.00019946, 0.0002991 , 0.00039892,
       0.00049868, 0.00039895, 0.00049865, 0.00039887, 0.00039904]), 'std_fit_time': array([0.00048829, 0.00048585, 0.00039892, 0.00045688, 0.00048858,
       0.00049868, 0.00048861, 0.00049865, 0.00048852, 0.00048872]), 'mean_score_time': array([0.00109742, 0.00149987, 0.00169795, 0.00169206, 0.00099704,
       0.00069826, 0.0006979 , 0.00099747, 0.00079784, 0.0010968 ]), 'std_score_time': array([5.37216173e-04, 5.02972611e-04, 4.62196945e-04, 1.00124460e-03,
       1.29794558e-05, 4.57116641e-04, 4.56882423e-04, 5.33653004e-07,
       3.98922508e-04, 2.99191836e-04]), 'param_weights': masked_array(data=['distance', 'uniform', 'uniform', 'uniform', 'uniform',
                   'distance', 'distance', 'uniform', 'distance',
                   'uniform'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
      

In [11]:
print('随机搜索-最佳度量值:',grid.best_score_)  # 获取最佳度量值
print('随机搜索-最佳参数：',grid.best_params_)  # 获取最佳度量值时的代定参数的值。是一个字典
print('随机搜索-最佳模型：',grid.best_estimator_)  # 获取最佳度量时的分类器模型

随机搜索-最佳度量值: 0.98
随机搜索-最佳参数： {'weights': 'uniform', 'n_neighbors': 18}
随机搜索-最佳模型： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=18, p=2,
                     weights='uniform')


In [10]:
# 使用获取的最佳参数生成模型，预测数据
knn = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'], weights=grid.best_params_['weights'])  # 取出最佳参数进行建模
knn.fit(X, y)  # 训练模型
print(knn.predict([[3, 5, 4, 2]]))  # 预测新对象

[1]


In [12]:
# =====================================自定义度量===========================================
from sklearn import metrics
# 自定义度量函数
def scorerfun(estimator, X, y):
    y_pred = estimator.predict(X)
    return metrics.accuracy_score(y, y_pred)

rand = RandomizedSearchCV(knn, param_grid, cv=10, 
                          scoring=scorerfun, n_iter=10, random_state=5)  #
rand.fit(X, y)

print('随机搜索-最佳度量值:',grid.best_score_)  # 获取最佳度量值

随机搜索-最佳度量值: 0.98


## 案例3 -- SVM GridSearchCV

In [16]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [17]:
# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
# 分别 取出 数据 与 标签 reshape的作用是把原始矩阵格式的像素数据转化为一行一个样本的形式
X = digits.images.reshape((n_samples, -1))
y = digits.target

# Split the dataset in two equal parts
# 分割测试数据与训练数据(注意这里已经分割了数据集)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [18]:
# Set the parameters by cross-validation
# 输入模型的超参由验证集来选择
# SVM主要的超参有类似于正则的系数和内核函数
tuned_parameters = [{
    'kernel': ['rbf'],
    'gamma': [1e-3, 1e-4],
    'C': [1, 10, 100, 1000]
    }, {
    'kernel': ['linear'],
    'C': [1, 10, 100, 1000]
    }]

#观察角度分别有准度与回归
scores = ['precision', 'recall']

In [19]:

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    # 通过 GridSearchCV 搜索最佳的超参数
    clf = GridSearchCV(SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
  # 这里进行交叉验证的数据是之前分割的训练数据
  # 而交叉验证本身又会分割数据，所以交叉验证这里分割的测试集我么可以看做为验证集，用来拟合模型的超参
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    # 注意最后利用测试集展示的才是泛化误差
    y_true, y_pred = y_test, clf.predict(X_test)
    
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.986 (+/-0.016) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.959 (+/-0.029) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.026) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.025) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.025) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.975 (+/-0.014) for {'C': 1, 'kernel': 'linear'}
0.975 (+/-0.014) for {'C': 10, 'kernel': 'linear'}
0.975 (+/-0.014) for {'C': 100, 'kernel': 'linear'}
0.975 (+/-0.014) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed o

[Sklearn参数详解—SVM](https://cloud.tencent.com/developer/article/1146077)

[sklearn参数](https://www.cnblogs.com/solong1989/p/9620170.html)